# LLama Index Implementation
### DB search using English (and ChatGPT)


In [21]:
from sqlalchemy import create_engine, text
import pandas as pd

In [22]:
engine = create_engine("postgresql+psycopg2:///?user=postgres&password=123@Team&host=127.0.0.1&port=5432&database=kmfl_llama")
# dbConnection = engine.connect()

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM tbl_kmfl WHERE open_whole_day = true"))
    for row in result:
        print(row)

(3, 'Al Shafic Nursing Home Limited', 'Al Shafic Nursing Home Limited', 'Level 3', 'NURSING HOME', 'Nursing Homes', 'Private Practice - Clinical Officer', 'Private Practice', 'Admitting  General Patients Only', 'Operational', True, True, True, True, 18, 5, "['24fd65c9-56ea-497c-82ec-3fdf0ac9d006', '36d2f7a8-41b3-40f8-8943-e513fa28c8db', 'f312d0b2-5e41-4587-b54d-ff1d1bc128c6', '4a5acc1b-894f-4405-b80e-8e1 ... (382 characters truncated) ... 1695-486e-85bd-4df0c4571b01', '3a7cb460-2ecb-450b-b21a-4c67e90ba6d2', '6f8c6c75-135c-4911-848d-8d3467c00605', '3907645f-f8f9-44da-b66c-ebf73bedacac']", "['1a536286-c5b9-46f4-a72f-919c1fccb8e8', '77e15030-e647-49ca-a4f8-8b6ed243343f', '175f9481-9e91-481a-aa47-173e790c41df', '498a9c31-96b7-4797-8731-f8f ... (502 characters truncated) ... ff66-47e2-8e7f-130be6ef0c9f', 'c4de260c-2818-46af-a1a1-cc0448768dc3', '8ebf7fc6-6a8d-4d1e-9a36-d68f59060dd4', '944f6655-7bfb-42c0-baee-5a62228f858a']", 40.044888, 1.7440495, 'Wajir', 'Wajir East', 'Wajir East', 'Barwago

In [23]:
from llama_index import SQLDatabase

table_details={
    'tbl_kmfl': 'Kenya facility list - list of hospitals, dispensaries, clinics etc',
    'service_category': 'categories of services offered in facilities',
    'service': 'services offered in facilities',
    'serv_cat_fac_mapper': 'service-category to facility mapper',
    'serv_fac_mapper': 'service to facility mapper',
    # 'pharmacies': 'pharmacies'
}

ze_model = 'gpt-3.5-turbo' #'gpt-3.5-turbo-instruct' # 

sql_db  = SQLDatabase(engine, sample_rows_in_table_info=2)

In [24]:
list(sql_db._all_tables)
# print(sql_db.table_info)

['tbl_kmfl',
 'serv_cat_fac_mapper',
 'service_category',
 'serv_fac_mapper',
 'service']

In [25]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPEN_AI_KEY")


In [26]:
import tiktoken
from llama_index.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model(ze_model).encode
)

callback_manager = CallbackManager([token_counter])

In [27]:
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI

llm = OpenAI(temperature=.1, model=ze_model)

service_ctx = ServiceContext.from_defaults(
    llm=llm, callback_manager=callback_manager
)

In [28]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_db,
    service_context=service_ctx
)


In [29]:
# query_str = "how many tables are we querying, what are the table names?"
# # response = query_engine.query(query_str)
# response  = query_engine.query('facilities that have child services; produce postgres sql using "like" statement where possible')

In [30]:
# print(response)
# print(response.metadata['sql_query'])
# print(token_counter.total_llm_token_count)

#### Efficient querying?

In [31]:
from llama_index.objects import ObjectIndex, SQLTableNodeMapping, SQLTableSchema

tables = list(sql_db._all_tables)
table_node_mapping = SQLTableNodeMapping(sql_db)
table_schema_objs=[]

for table in tables:
    table_schema_objs.append(SQLTableSchema(table_name=table, context_str=table_details[table]))

In [32]:
table_schema_objs

[SQLTableSchema(table_name='tbl_kmfl', context_str='Kenya facility list - list of hospitals, dispensaries, clinics etc'),
 SQLTableSchema(table_name='serv_cat_fac_mapper', context_str='service-category to facility mapper'),
 SQLTableSchema(table_name='service_category', context_str='categories of services offered in facilities'),
 SQLTableSchema(table_name='serv_fac_mapper', context_str='service to facility mapper'),
 SQLTableSchema(table_name='service', context_str='services offered in facilities')]

In [33]:
from llama_index.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index import VectorStoreIndex

obj_index = ObjectIndex.from_objects(
    table_schema_objs, table_node_mapping, VectorStoreIndex, service_context = service_ctx
)

query_engine = SQLTableRetrieverQueryEngine(sql_db, obj_index.as_retriever(similarity_top_k=3), service_context=service_ctx)



In [41]:
resp = query_engine.query('facilities that have many cots; produce postgres-sql using "like" statement where possible')
# ; in the produced sql query replace "like" with "ilike"

print('response: ', resp)
print('metadata: ', resp.metadata['sql_query'])
# print('metadata: ', resp.metadata['result'])
print('tokens: ', token_counter.total_llm_token_count)

response:  The facilities that have many cots are:
1. Kenyatta National Hospital - 427 cots
2. Pumwani Maternity Hospital - 150 cots
3. Tenwek Mission Hospital - 100 cots
4. Nairobi Hospital - 100 cots
5. Embu Level 5 Hospital - 87 cots
6. Kakamega County General Hospital - 79 cots
7. New Busia Maternity & Nursing Home - 78 cots
8. St Mulumba Mission Hospital - 70 cots
9. Nakuru Provincial General Hospital - 68 cots
10. Moi Teaching Referral Hospital - 63 cots

And so on...
metadata:  SELECT name, cots
FROM tbl_kmfl
WHERE cots > 10
ORDER BY cots DESC;
tokens:  8424
